## Laboratorium 2 
**Bartosz Więcek**   
\
Zaimportowanie potrzebnych bibliotek:

In [1]:
import pandas as pd
import numpy as np

\
Otworzenie pliku z danymi z równoczesną podmianą nazw kolumn na bardziej zrozumiałe:

In [2]:
new_column_names = ['country', 'year', 'total_cases', 'm04', 'm514', 'm014', 'm1524', 'm2534', 'm3544', 'm4554', 'm5564', 'm65', 'mu', 'f04', 'f514', 'f014', 'f1524', 'f2534', 'f3544', 'f4554', 'f5564', 'f65', 'fu']
d = pd.read_csv('../Original Data/tb.csv', names=new_column_names, header=0)

\
Wejściowe dane:

In [3]:
d

,country,year,total_cases,m04,m514,m014,m1524,m2534,m3544,m4554,...,f04,f514,f014,f1524,f2534,f3544,f4554,f5564,f65,fu
0,AD,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AD,1993,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5764,ZW,2004,14581.0,NaN,NaN,187.0,833.0,2908.0,2298.0,1056.0,...,NaN,NaN,225.0,1140.0,2858.0,1565.0,622.0,214.0,111.0,NaN
5765,ZW,2005,13155.0,NaN,NaN,210.0,837.0,2264.0,1855.0,762.0,...,NaN,NaN,269.0,1136.0,2242.0,1255.0,578.0,193.0,603.0,NaN
5766,ZW,2006,12718.0,NaN,NaN,215.0,736.0,2391.0,1939.0,896.0,...,NaN,NaN,237.0,1020.0,2424.0,1355.0,632.0,230.0,96.0,NaN
5767,ZW,2007,10583.0,6.0,132.0,138.0,500.0,3693.0,0.0,716.0,...,7.0,178.0,185.0,739.0,3311.0,0.0,553.0,213.0,90.0,NaN


\
Przeprowadzenie meltingu:

In [4]:
d1 = pd.melt(d, id_vars =['country', 'year'], value_vars =new_column_names[3:],
              var_name ='column', value_name ='cases')
d1.sort_values(['country', 'year'], inplace=True)
d1.reset_index(drop=True, inplace=True)

d1

,country,year,column,cases
0,AD,1989,m04,NaN
1,AD,1989,m514,NaN
2,AD,1989,m014,NaN
3,AD,1989,m1524,NaN
4,AD,1989,m2534,NaN
...,...,...,...,...
115375,NaN,2008,f3544,437.0
115376,NaN,2008,f4554,226.0
115377,NaN,2008,f5564,110.0
115378,NaN,2008,f65,80.0


\
Pozbycie się wierszy zawierających puste komórki (NaN):

In [5]:
d1 = d1.loc[d1['cases'] >= 0]
d1.reset_index(drop=True, inplace=True)
d1= d1[:35552]

d1

,country,year,column,cases
0,AD,1996,m014,0.0
1,AD,1996,m1524,0.0
2,AD,1996,m2534,0.0
3,AD,1996,m3544,4.0
4,AD,1996,m4554,1.0
...,...,...,...,...
35547,ZW,2008,f3544,2890.0
35548,ZW,2008,f4554,467.0
35549,ZW,2008,f5564,174.0
35550,ZW,2008,f65,105.0


\
Funkcja pomagająca wyciągnąć z nazw kolumn przedziały wiekowe:

In [6]:
def make_age_more_clear(age_as_str):       
    if len(age_as_str) == 1 or age_as_str == '65':
        return '65+'
    elif len(age_as_str) == 2:
        return age_as_str[0] + '--' + age_as_str[1]
    
    elif len(age_as_str) == 3:
        return age_as_str[0] + '--' + age_as_str[1:]
    
    elif len(age_as_str) == 4:
        return age_as_str[0: 2] + '--' + age_as_str[2:]
    
    else:
        raise ValueError('Undefined age')

\
Zmienna zabezpieczająca przed próbami działania na nieistniejących kolumnach (np. po zmianie nazwy):

In [7]:
change_made = False

\
Rozbicie kolumny 'column' zawierającej informacje o płci i wieku na dwie osobne kolumny 'sex' oraz 'age':

In [9]:
if not change_made:
    d1 = d1.rename(columns={'column': 'age'})
    sex_tab = []

    for i in range(d1.shape[0]):
        sex_with_age = d1['age'][i]
        sex_tab.append(sex_with_age[0])
        d1['age'][i] = make_age_more_clear(sex_with_age[1:])

    d1.insert(2, 'sex', sex_tab)
    
    change_made = True

\
Dane uporządkowane zgodnie z zasadami 'tidy data':

In [10]:
d1

,country,year,sex,age,cases
0,AD,1996,m,0--14,0.0
1,AD,1996,m,15--24,0.0
2,AD,1996,m,25--34,0.0
3,AD,1996,m,35--44,4.0
4,AD,1996,m,45--54,1.0
...,...,...,...,...,...
35547,ZW,2008,f,35--44,2890.0
35548,ZW,2008,f,45--54,467.0
35549,ZW,2008,f,55--64,174.0
35550,ZW,2008,f,65+,105.0


\
Zapisanie przetworzonych danych do pliku processed_tb.csv w katalogu Analysis Data:

In [10]:
d1.to_csv('../Analysis Data/processed_tb.csv', index=False)